# Alkım Ege Akarsu | 21901461 | GE 461 | Project 5: Data Stream Mining

## Import Modules

In [ ]:
from pathlib import Path
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skmultiflow.data import AGRAWALGenerator, SEAGenerator, DataStream
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.meta import AdaptiveRandomForestClassifier, DynamicWeightedMajorityClassifier
from skmultiflow.lazy import SAMKNNClassifier
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.drift_detection import ADWIN
from skmultiflow.anomaly_detection import HalfSpaceTrees

SEED = 0

## a) Dataset Preparation

In [ ]:
# # Get the stream objects
# agr_gens = [AGRAWALGenerator(classification_function=i, random_state=SEED) for i in range(4)]
# sea_gens = [SEAGenerator(classification_function=i, random_state=SEED) for i in range(4)]

# # Generate datasets
# agr_datasets = [agr_gen.next_sample(25000) for agr_gen in agr_gens]
# sea_datasets = [sea_gen.next_sample(25000) for sea_gen in sea_gens]

# # Combine datasets
# agr_dataset = (np.vstack([d[0] for d in agr_datasets]), np.hstack([d[1] for d in agr_datasets]))
# sea_dataset = (np.vstack([d[0] for d in sea_datasets]), np.hstack([d[1] for d in sea_datasets]))

# # Merge X and y from tuples
# agr_X = agr_dataset[0]
# agr_y = agr_dataset[1]
# sea_X = sea_dataset[0]
# sea_y = sea_dataset[1]
# agr_Xy = np.hstack((agr_X, agr_y.reshape(-1, 1)))
# sea_Xy = np.hstack((sea_X, sea_y.reshape(-1, 1)))

# # Convert to DataFrame
# agr_column_names = agr_gens[0].feature_names + agr_gens[0].target_names
# agr_df = pd.DataFrame(agr_Xy, columns=agr_column_names)
# sea_column_names = sea_gens[0].feature_names + sea_gens[0].target_names
# sea_df = pd.DataFrame(sea_Xy, columns=sea_column_names)

# # Save datasets
# agr_df.to_csv(Path().resolve().joinpath("AGRAWALDataset.csv"), index=False)
# sea_df.to_csv(Path().resolve().joinpath("SEADataset.csv"), index=False)

## a) Load Datasets

In [ ]:
# Load datasets
agr_dataset = pd.read_csv(Path().resolve().joinpath("AGRAWALDataset.csv"), dtype={"target": np.uint8})
sea_dataset = pd.read_csv(Path().resolve().joinpath("SEADataset.csv"), dtype={"target_0": np.uint8})
spam_dataset = pd.read_csv(Path().resolve().joinpath("SPAMDataset.csv"), dtype={"target": np.uint8})
elec_dataset = pd.read_csv(Path().resolve().joinpath("ELECTRICITYDataset.csv"), dtype={"target": np.uint8})

# Separate X and y
agr_X = agr_dataset.drop(columns=["target"])
agr_y = agr_dataset["target"].to_numpy().reshape(-1, 1)
sea_X = sea_dataset.drop(columns=["target_0"])
sea_y = sea_dataset["target_0"].to_numpy().reshape(-1, 1)
spam_X = spam_dataset.drop(columns=["target"])
spam_y = spam_dataset["target"].to_numpy().reshape(-1, 1)
elec_X = elec_dataset.drop(columns=["target"])
elec_y = elec_dataset["target"].to_numpy().reshape(-1, 1)

# Get stream instances
agr_stream = DataStream(agr_dataset)
sea_stream = DataStream(sea_dataset)
spam_stream = DataStream(spam_dataset)
elec_stream = DataStream(elec_dataset)

# Get distribution of labels in each dataset
# AGR
num_0 = agr_dataset["target"].value_counts()[0]
num_total = agr_dataset["target"].count()
print(f"Agrawal Dataset ({num_total} data points):\n"
      f"Percentage of 0's: {(num_0 / num_total) * 100:.2f}%\n"
      f"Percentage of 1's: {(100 - ((num_0 / num_total) * 100)):.2f}%\n")
# SEA
num_0 = sea_dataset["target_0"].value_counts()[0]
num_total = sea_dataset["target_0"].count()
print(f"SEA Dataset ({num_total} data points):\n"
      f"Percentage of 0's: {(num_0 / num_total) * 100:.2f}%\n"
      f"Percentage of 1's: {(100 - ((num_0 / num_total) * 100)):.2f}%\n")
# SPAM
num_0 = spam_dataset["target"].value_counts()[0]
num_total = spam_dataset["target"].count()
print(f"Spam Dataset ({num_total} data points):\n"
      f"Percentage of 0's: {(num_0 / num_total) * 100:.2f}%\n"
      f"Percentage of 1's: {(100 - ((num_0 / num_total) * 100)):.2f}%\n")
# ELEC
num_0 = elec_dataset["target"].value_counts()[0]
num_total = elec_dataset["target"].count()
print(f"Electricity Dataset ({num_total} data points):\n"
      f"Percentage of 0's: {(num_0 / num_total) * 100:.2f}%\n"
      f"Percentage of 1's: {(100 - ((num_0 / num_total) * 100)):.2f}%\n")

## b.1)

### Functions

In [ ]:
def run_algo_b1(learner, dataset_y, model_name, dataset_name, stream, n_warm_start=200, test_interval=1000):
    # Prepare the stream
    stream.restart()
    # Prepare storages
    pred_y = np.empty([test_interval])
    accs = {}
    mean_accs = {}

    # Warm start (Train for the first "n_warm_start" samples)
    for i in range(n_warm_start):
        # Get new sample
        X, y = stream.next_sample()
        
        # Partial fit
        learner.partial_fit(X, y, classes=[0, 1])

    # Main loop
    for i in range(stream.n_remaining_samples()):
        # Calculate dataset index
        dataset_idx = i + n_warm_start
        # Get new sample
        X, y = stream.next_sample()
        
        # Predict the new sample
        pred_idx = i % test_interval
        pred_y[pred_idx] = learner.predict(X)[0]
        
        # Partial fit
        learner.partial_fit(X, y, classes=[0, 1])
        
        # Report test acc every "test_interval" samples
        if (i + 1) % test_interval == 0:
            # Get acc
            start_idx = (dataset_idx + 1) - test_interval
            end_idx = dataset_idx + 1
            acc = (pred_y == dataset_y[start_idx:end_idx].squeeze()).mean()
            # Store acc
            accs[dataset_idx] = acc
            # Get mean of accs so far
            mean_accs[dataset_idx] = np.mean(list(accs.values()))
            # Print progress
            print(f"At sample: {dataset_idx}", flush=True)
    
    # Print final mean accuracy
    print(f"Final Mean Accuracy: {list(mean_accs.values())[-1]:.3f}")

    # Plot the results
    plt.figure()
    plt.suptitle(f"{model_name}", fontsize=16, fontweight="bold")
    plt.title(f"{dataset_name} Dataset", fontsize=14)
    plt.xlabel("Samples")
    plt.ylabel("Accuracy")
    plt.ylim(0.5, 1)
    plt.plot(list(accs.keys()), list(accs.values()), label=f"Past {test_interval} Sample Accuracy")
    plt.plot(list(mean_accs.keys()), list(mean_accs.values()), label="Mean Accuracy")
    plt.legend()
    plt.show()

In [ ]:
# Get classifier instances
arf = AdaptiveRandomForestClassifier()
samknn = SAMKNNClassifier()
dwm = DynamicWeightedMajorityClassifier()

### Agrawal Dataset

In [ ]:
run_algo_b1(arf, agr_y, "Adaptive Random Forest", "Agrawal", agr_stream)
run_algo_b1(samknn, agr_y, "Streaming Agnostic Model with k-Nearest Neighbor", "Agrawal", agr_stream)
run_algo_b1(dwm, agr_y, "Dynamic Weighted Majority", "Agrawal", agr_stream)

### SEA Dataset

In [ ]:
run_algo_b1(arf, sea_y, "Adaptive Random Forest", "SEA", sea_stream)
run_algo_b1(samknn, sea_y, "Streaming Agnostic Model with k-Nearest Neighbor", "SEA", sea_stream)
run_algo_b1(dwm, sea_y, "Dynamic Weighted Majority", "SEA", sea_stream)

### Spam Dataset

In [ ]:
run_algo_b1(arf, spam_y, "Adaptive Random Forest", "Spam", spam_stream, test_interval=62)
run_algo_b1(samknn, spam_y, "Streaming Agnostic Model with k-Nearest Neighbor", "Spam", spam_stream, test_interval=62)
run_algo_b1(dwm, spam_y, "Dynamic Weighted Majority", "Spam", spam_stream, test_interval=62)

### Electricity Dataset

In [ ]:
run_algo_b1(arf, elec_y, "Adaptive Random Forest", "Electricity", elec_stream, test_interval=453)
run_algo_b1(samknn, elec_y, "Streaming Agnostic Model with k-Nearest Neighbor", "Electricity", elec_stream, test_interval=453)
run_algo_b1(dwm, elec_y, "Dynamic Weighted Majority", "Electricity", elec_stream, test_interval=453)

## b.2)

### Functions

In [ ]:
def partial_fit(X, y, learners):
    # Partial fit each model with prob "model_fit_prob"
    for learner in learners:
        for _ in range(np.random.poisson(1)):
            learner.partial_fit(X, y, classes=[0, 1])


def predict(X, learners):
    # Predictions storage
    learner_preds_proba = np.empty([X.shape[0], len(learners)])
    preds = np.empty([X.shape[0]])
    
    # Predict for each learner
    for i, learner in enumerate(learners):
        try:
            learner_preds_proba[:, i] = learner.predict_proba(X)[0][1]
        except:
            learner_preds_proba[:, i] = learner.predict_proba(X)[0][0]
            print(f"Error in prediction, value: {learner_preds_proba[:, i]}", flush=True)
        
    # Get mean probability for each sample
    preds = np.mean(learner_preds_proba, axis=1)
    
    # Probas to labels
    preds[preds > 0.5] = 1
    preds[preds <= 0.5] = 0
    
    return preds


def run_algo_b2(n_learners, dataset_y, dataset_name, stream, n_warm_start=200, test_interval=1000):
    # Prepare base learners
    learners = [HoeffdingTreeClassifier() for _ in range(n_learners)]
    # Prepare the stream
    stream.restart()
    # Prepare hyperparameters
    just_reset = False
    # Prepare drift detector
    drift_detector = ADWIN()
    # Prepare storages
    pred_y = np.empty([test_interval])
    accs = {}
    mean_accs = {}

    # Warm start (Train for the first "n_warm_start" samples)
    for i in range(n_warm_start):
        # Get new sample
        X, y = stream.next_sample()
        
        # Partial fit each model (online bagging)
        partial_fit(X, y, learners)
        
        # Add sample to drift detector
        drift_detector.add_element(np.float64(y))

    # Main loop
    for i in range(stream.n_remaining_samples()):
        # Calculate dataset index
        dataset_idx = i + n_warm_start
        # Get new sample
        X, y = stream.next_sample()
        
        # Predict the new sample
        pred_idx = i % test_interval
        pred_y[pred_idx] = predict(X, learners)
        
        # Add sample to drift detector
        drift_detector.add_element(np.float64(y))
        # Check for drift
        if drift_detector.detected_change():
            # Set reset flag
            just_reset = True
            # Print alert
            print(f"Drift detected, index {dataset_idx}", flush=True)
            # Reset learners
            for learner in learners:
                learner.reset()
            # Reset drift detector
            drift_detector.reset()
        
        # Partial fit each model (online bagging)
        if just_reset:
            for learner in learners:
                learner.partial_fit(X, y, classes=[0, 1])
            just_reset = False
        else:
            partial_fit(X, y, learners)
        
        # Report test acc every "test_interval" samples
        if (i + 1) % test_interval == 0:
            # Get acc
            start_idx = (dataset_idx + 1) - test_interval
            end_idx = dataset_idx + 1
            acc = (pred_y == dataset_y[start_idx:end_idx].squeeze()).mean()
            # Store acc
            accs[dataset_idx] = acc
            # Get mean of accs so far
            mean_accs[dataset_idx] = np.mean(list(accs.values()))
            # Print progress
            print(f"At sample: {dataset_idx}", flush=True)
    
    # Print final mean accuracy
    print(f"Final Mean Accuracy: {list(mean_accs.values())[-1]:.3f}")

    # Plot the results
    plt.figure()
    plt.suptitle("Ensemble Model", fontsize=16, fontweight="bold")
    plt.title(f"{dataset_name} Dataset", fontsize=14)
    plt.xlabel("Samples")
    plt.ylabel("Accuracy")
    plt.ylim(0.5, 1)
    plt.plot(list(accs.keys()), list(accs.values()), label=f"Past {test_interval} Sample Accuracy")
    plt.plot(list(mean_accs.keys()), list(mean_accs.values()), label="Mean Accuracy")
    plt.legend()
    plt.show()

In [ ]:
run_algo_b2(10, agr_y, "Agrawal", agr_stream)
run_algo_b2(10, sea_y, "SEA", sea_stream)
run_algo_b2(10, spam_y, "Spam", spam_stream, test_interval=62)
run_algo_b2(10, elec_y, "Electricity", elec_stream, test_interval=453)

## c.1)

In [ ]:
# Generate indices for label flipping
corrupt_idx_1 = np.random.choice(list(range(10000, 10501)), size=50, replace=False)
corrupt_idx_2 = np.random.choice(list(range(20000, 20501)), size=100, replace=False)
corrupt_idx = np.hstack([corrupt_idx_1, corrupt_idx_2]).squeeze()

# Generate corrupted datasets
agr_corrupted = deepcopy(agr_dataset)
agr_corrupted.loc[corrupt_idx, "target"] = 1 - agr_corrupted.loc[corrupt_idx, "target"]
sea_corrupted = deepcopy(sea_dataset)
sea_corrupted.loc[corrupt_idx, "target_0"] = 1 - sea_corrupted.loc[corrupt_idx, "target_0"]

# Generate corrupted streams
agr_corrupted_stream = DataStream(agr_corrupted.iloc[:25000,:])
sea_corrupted_stream = DataStream(sea_corrupted)

## c.2)

### Functions

In [ ]:
def partial_fit(X, y, learners):
    # Partial fit each model with prob "model_fit_prob"
    for learner in learners:
        for _ in range(np.random.poisson(1)):
            learner.partial_fit(X, y, classes=[0, 1])


def predict(X, learners):
    # Predictions storage
    learner_preds_proba = np.empty([X.shape[0], len(learners)])
    preds = np.empty([X.shape[0]])
    
    # Predict for each learner
    for i, learner in enumerate(learners):
        try:
            learner_preds_proba[:, i] = learner.predict_proba(X)[0][1]
        except:
            learner_preds_proba[:, i] = learner.predict_proba(X)[0][0]
            print(f"Error in prediction, value: {learner_preds_proba[:, i]}", flush=True)
        
    # Get mean probability for each sample
    preds = np.mean(learner_preds_proba, axis=1)
    
    # Probas to labels
    preds[preds > 0.5] = 1
    preds[preds <= 0.5] = 0
    
    return preds


def add_to_window(window, X):
    # Add sample to window
    window = np.delete(window, 0, axis=0)
    window = np.vstack((window, X))
    return window


def is_outlier(X, window, threshold=3):
    # Get mean and std of each feature
    mean = np.mean(window, axis=0)
    std = np.std(window, axis=0)
    
    # Get the mean absolute z score of each feature of X
    z_score = np.max(np.abs((X - mean) / std))
    
    # Make a decision
    return z_score > threshold


def run_algo_c2(n_learners, dataset_y, dataset_name, stream, n_warm_start=200, test_interval=1000):
    # Prepare base learners
    learners = [HoeffdingTreeClassifier() for _ in range(n_learners)]
    # Prepare the stream
    stream.restart()
    # Prepare drift detector
    drift_detector = ADWIN()
    just_reset = False
    # Prepare outlier detector
    z_score_window_size = 200
    window_0 = np.zeros((z_score_window_size, stream.n_features))
    window_1 = np.zeros((z_score_window_size, stream.n_features))
    outlier_z_score_threshold = 3.2
    sample_outlier = False
    outliers = []
    # Prepare storages
    pred_y = np.empty([test_interval])
    accs = {}
    mean_accs = {}

    # Warm start (Train for the first "n_warm_start" samples)
    for i in range(n_warm_start):
        # Get new sample
        X, y = stream.next_sample()
        
        # Add to window
        if y[0] == 0:
            window_0 = add_to_window(window_0, X)
        else:
            window_1 = add_to_window(window_1, X)
        
        # Partial fit each model (online bagging)
        partial_fit(X, y, learners)
        
        # Add sample to drift detector
        drift_detector.add_element(np.float64(y))

    # Main loop
    for i in range(stream.n_remaining_samples()):
        # Calculate dataset index
        dataset_idx = i + n_warm_start
        # Get new sample
        X, y = stream.next_sample()
        
        # Check for outlier
        if y[0] == 0:
            sample_outlier = is_outlier(X, window_0, outlier_z_score_threshold)
            if sample_outlier == False:
                window_0 = add_to_window(window_0, X)
        else:
            sample_outlier = is_outlier(X, window_1, outlier_z_score_threshold)
            if sample_outlier == False:
                window_1 = add_to_window(window_1, X)
        
        # Predict the new sample
        pred_idx = i % test_interval
        pred_y[pred_idx] = predict(X, learners)
        
        # Add sample to drift detector
        drift_detector.add_element(np.float64(y))
        # Check for drift
        if drift_detector.detected_change():
            # Set reset flag
            just_reset = True
            # Print alert
            print(f"Drift detected, index {dataset_idx}", flush=True)
            # Reset learners
            for learner in learners:
                learner.reset()
            # Reset drift detector
            drift_detector.reset()
            
        # Partial fit each model (online bagging)
        if just_reset:
            for learner in learners:
                learner.partial_fit(X, y, classes=[0, 1])
            just_reset = False
        else:
            if sample_outlier == False:
                partial_fit(X, y, learners)
            else:
                # Add to list
                print(f"Outlier detected, index {dataset_idx}", flush=True)
                outliers.append(dataset_idx)
                sample_outlier = False
        
        # Report test acc every "test_interval" samples
        if (i + 1) % test_interval == 0:
            # Get acc
            start_idx = (dataset_idx + 1) - test_interval
            end_idx = dataset_idx + 1
            acc = (pred_y == dataset_y[start_idx:end_idx].squeeze()).mean()
            # Store acc
            accs[dataset_idx] = acc
            # Get mean of accs so far
            mean_accs[dataset_idx] = np.mean(list(accs.values()))
            # Print progress
            print(f"At sample: {dataset_idx}", flush=True)
    
    # Print final mean accuracy
    print(f"Final Mean Accuracy: {list(mean_accs.values())[-1]:.3f}")

    # Plot the results
    plt.figure()
    plt.suptitle("Ensemble Model With Outlier Detection", fontsize=16, fontweight="bold")
    plt.title(f"{dataset_name} Dataset", fontsize=14)
    plt.xlabel("Samples")
    plt.ylabel("Accuracy")
    plt.ylim(0.5, 1)
    plt.plot(list(accs.keys()), list(accs.values()), label=f"Past {test_interval} Sample Accuracy")
    plt.plot(list(mean_accs.keys()), list(mean_accs.values()), label="Mean Accuracy")
    for idx in outliers:
        plt.axvline(idx, color="r", alpha=0.1)
    plt.legend()
    plt.show()

In [ ]:
run_algo_c2(10, agr_y[:25000,:], "Agrawal", agr_corrupted_stream, test_interval=250)
# run_algo_c2(10, sea_y, "SEA", sea_corrupted_stream)
# run_algo_b2(10, agr_y, "Agrawal", agr_corrupted_stream)
# run_algo_b2(10, sea_y, "SEA", sea_corrupted_stream)